# Programa que prepara o dataset para o Neo4J

_Este arquivo foi utilizado para criar um grafo menor, a partir dos dados incompletos que tínhamos gerado até a etapa de prévia do projeto. Ele nos possibilitou visualizar melhor alguns resultados, por ser um recorte do outro grafo._

## Importando bibliotecas

In [1]:
import pandas as pd

## Carregando nossos datasets

In [2]:
atores = pd.read_csv("atores.csv")
diretores = pd.read_csv("diretores.csv")
generos = pd.read_csv("generos.csv")

## Criando csvs para o Neo4j

In [36]:
actors = []
directors = []
midias = []
genres = []

for i in range(len(atores)):
    midias.append(str(atores['Titulo'][i]) + " (" + str(atores['Ano'][i] + ")"))
    actors.append(str(atores['Ator'][i]))

table = {'Atores': actors, 'Titulo': midias}
df = pd.DataFrame(table, columns=['Titulo', 'Atores'])
df.to_csv(r'tituloatores.csv',
            index=False, header=True)

midias = []

for i in range(len(diretores)):
    midias.append(str(diretores['Titulo'][i]) + " (" + str(diretores['Ano'][i] + ")"))
    directors.append(str(diretores['Diretor'][i]))

table = {'Titulo': midias, 'Diretores': directors}
df = pd.DataFrame(table, columns=['Titulo', 'Diretores'])
df.to_csv(r'titulodiretores.csv',
            index=False, header=True)

midias = []

for i in range(len(generos)):
    midias.append(str(generos['Titulo'][i]) + " (" + str(generos['Ano'][i] + ")"))
    genres.append(str(generos['Genero'][i]))

table = {'Titulo': midias, 'Generos': genres}
df = pd.DataFrame(table, columns=['Titulo', 'Generos'])
df.to_csv(r'titulogeneros.csv',
            index=False, header=True)

midias = []
for i in range(len(generos)):
    if (str(generos['Titulo'][i]) + " (" + str(generos['Ano'][i] + ")")) not in midias:
        midias.append(str(generos['Titulo'][i]) + " (" + str(generos['Ano'][i] + ")"))

table = {'Titulo': midias}
df = pd.DataFrame(table, columns=['Titulo'])
df.to_csv(r'titulos.csv',
            index=False, header=True)

actors = []
for i in range(len(atores)):
    if (str(atores['Ator'][i])) not in actors and str(atores['Ator'][i]) != "nan" and str(atores['Ator'][i]) != "":
        actors.append(atores['Ator'][i])

table = {'Atores': actors}
df = pd.DataFrame(table, columns=['Atores'])
df.to_csv(r'apenasatores.csv',
            index=False, header=True)

directors = []
for i in range(len(diretores)):
    if (str(diretores['Diretor'][i])) not in directors and str(diretores['Diretor'][i]) != "nan" and str(diretores['Diretor'][i]) != "":
        directors.append(diretores['Diretor'][i])

table = {'Diretores': directors}
df = pd.DataFrame(table, columns=['Diretores'])
df.to_csv(r'apenasdiretores.csv',
            index=False, header=True)

# Comandos para Cypher

## Criando os nós e as arestas
_Obs: os arquivos cujos links apontam para o github também estão na pasta data do projeto_

### Criando nós
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/apenasatores.csv' AS line
CREATE (a:Ator {Ator: line.Atores})

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/apenasdiretores.csv' AS line
CREATE (d:Diretor {Diretor: line.Diretores})

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/titulogeneros.csv' AS line
Merge (g:Genero {Genero: line.Generos})

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/titulos.csv' AS line
CREATE (t:Titulo {Titulo: line.Titulo})

### Criando arestas
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/tituloatores.csv' AS line
MATCH (t:Titulo {Titulo: line.Titulo})
MATCH (a:Ator {Ator: line.Atores})
CREATE (a)-[:Atuou]->(t)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/titulodiretores.csv' AS line
MATCH (t:Titulo {Titulo: line.Titulo})
MATCH (d:Diretor {Diretor: line.Diretores})
CREATE (d)-[:Dirigiu]->(t)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/GeorgeJuniorGG/git_test/main/titulogeneros.csv' AS line
MATCH (t:Titulo {Titulo: line.Titulo})
MATCH (g:Genero {Genero: line.Generos})
CREATE (g)-[:Genero]->(t)

### Criando arestas para projeções propostas

MATCH (p:Diretor)-[:Dirigiu]->(t:Titulo)<-[:Genero]-(g:Genero) 
MERGE (p)-[:AtuouGenero]->(g)

MATCH (p:Ator)-[:Atuou]->(t:Titulo)<-[:Genero]-(g:Genero)
MERGE (p)-[:AtuouGenero]->(g)

MATCH (p:Ator)-[:Atuou]->(t:Titulo)<-[:Atuou]-(q:Ator)
MERGE (p)-[:CoAtuou]-(q)